In [0]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib

In [0]:
from urllib.request import urlopen
page = urlopen('https://en.wikipedia.org/wiki/List_of_universities_in_England')
soup = BeautifulSoup(page, 'html.parser')

In [0]:
def fetchUniDetails(web_page):
    prefix="https://en.wikipedia.org"

    page2 = urlopen(prefix+web_page)
    soup2 = BeautifulSoup(page2, 'html.parser')
    uni_details=[None,None,None,None]
    
    
    table=soup2.find('table', class_='infobox vcard')
    try:
      for row in table.tbody.findAll('tr'):              
        if row.find(text=True)=='Former names' or row.find(text=True)=='Former name':
          if row.td.find(text=True)!='See History':
            uni_details[0]=row.td.find(text=True)

        if row.find(text=True)=='Students':
          uni_details[1]=row.td.find(text=True)

        if row.find(text=True)=='Undergraduates':
          uni_details[2]=row.td.find(text=True)
          
        if row.find(text=True)=='Postgraduates':
          uni_details[3]=row.td.find(text=True)

    except:
        pass
    return uni_details

In [0]:
university=[]
location=[]
students=[]
former_names=[]
undergraduates=[]
postgraduates=[]
established=[]

right_table=soup.find('table', class_='wikitable sortable')
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==6:
        university.append(cells[0].find(text=True))
        uni_url=cells[0].find('a',href=True)['href']
        location.append(cells[1].find(text=True))        
        established.append(cells[2].find(text=True))
        
        uni_details = fetchUniDetails(uni_url)
        former_names.append(uni_details[0] )
        if uni_details[1] is not None:
          students.append(uni_details[1])
        else:
          students.append(cells[3].find(text=True))
        undergraduates.append(uni_details[2])
        postgraduates.append(uni_details[3]  ) 


In [0]:
df=pd.DataFrame(university,columns=['university'])
df['former names']=former_names
df['students']=students
df['location']=location
df['undergraduates']=undergraduates
df['postgraduates']=postgraduates
df['established']=established
df.head()

,university,former names,students,location,undergraduates,postgraduates,established
0,AECC University College,None,-\n,Bournemouth,470,265,– (1965)\n
1,Anglia Ruskin University,None,"24,490 (2018/19)",Cambridge,"19,630 (2018/19)","4,860 (2018/19)",1992 (1858)\n
2,Arden University,Resource Development International,"8,000",Coventry,None,None,2015 (1990)\n
3,Arts University Bournemouth,"Arts University College at Bournemouth (AUCB),...","3,385 (2018/19)",Poole,"3,240 (2018/19)",150 (2018/19),2012 (1885)\n
4,University of the Arts London,London Institute,"18,290",London\n,"14,760 (2016/17)","3,530 (2016/17)",2004\n


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   university      130 non-null    object
 1   former names    43 non-null     object
 2   students        130 non-null    object
 3   location        130 non-null    object
 4   undergraduates  120 non-null    object
 5   postgraduates   121 non-null    object
 6   established     130 non-null    object
dtypes: object(7)
memory usage: 7.2+ KB


In [0]:
df[df['postgraduates'].isnull()]

,university,former names,students,location,undergraduates,postgraduates,established
2,Arden University,Resource Development International,"8,000",Coventry,None,None,2015 (1990)\n
15,BPP University,BPP Law School,"16,000",Abingdon,None,None,2013 (1992)\n
28,Cranfield University,Cranfield Institute of Technology,"4,490 (2018/19)",Cranfield,None,None,1993 (1946)\n
38,University College of Estate Management,None,–\n,Reading,None,None,– (1919)\n
41,University of Gloucestershire,None,"9,220 (2017/18)",Cheltenham,None,None,2001 (1998)\n
44,Hartpury University,None,"3,600",Gloucester,None,None,2018 (1947)\n
82,London Institute of Banking & Finance,None,-\n,London,None,None,– (1879)\n
104,Regent's University London,Regent's College,"3,772",London,None,None,2013 (1984)\n
127,Writtle University College,None,750,Chelmsford,None,None,– (1893)\n


In [0]:
df.fillna('Unavailable', inplace=True)

In [0]:
for col in ['students','undergraduates','postgraduates']:
  df[col]=df[col].apply(lambda x: x.split(' ')[0])
  df[col]=df[col].apply(lambda x: x.replace('-','na'))

for col in ['established','students','location']:
  df[col]=df[col].apply(lambda x: x.replace('\n',''))


In [0]:
df.head()

,university,former names,students,location,undergraduates,postgraduates,established
0,AECC University College,Unavailable,na,Bournemouth,470,265,– (1965)
1,Anglia Ruskin University,Unavailable,"24,490",Cambridge,"19,630","4,860",1992 (1858)
2,Arden University,Resource Development International,"8,000",Coventry,Unavailable,Unavailable,2015 (1990)
3,Arts University Bournemouth,"Arts University College at Bournemouth (AUCB),...","3,385",Poole,"3,240",150,2012 (1885)
4,University of the Arts London,London Institute,"18,290",London,"14,760","3,530",2004
